In [15]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd 


In [16]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 24C6-591C

 Directory of C:\Users\ninja\Desktop\land cover image classification

23.06.2024  22:12    <DIR>          .
23.06.2024  22:12    <DIR>          ..
23.06.2024  20:22    <DIR>          .ipynb_checkpoints
23.06.2024  20:21    <DIR>          AnnualCrop
23.06.2024  20:21    <DIR>          Forest
23.06.2024  20:21    <DIR>          HerbaceousVegetation
23.06.2024  20:21    <DIR>          Highway
23.06.2024  20:22    <DIR>          Industrial
23.06.2024  20:22    <DIR>          Pasture
23.06.2024  20:22    <DIR>          PermanentCrop
23.06.2024  20:22    <DIR>          Residential
23.06.2024  20:22    <DIR>          River
23.06.2024  20:23    <DIR>          SeaLake
23.06.2024  22:12            16.422 Untitled.ipynb
               1 File(s)         16.422 bytes
              13 Dir(s)     229.986.304 bytes free


In [17]:
dataset_path = "C:/Users/ninja/Desktop/land cover image classification"

In [18]:
from PIL import Image
import os

def is_image_file(filepath):
    try:
        img = Image.open(filepath)
        img.verify()   
        return True
    except (IOError, SyntaxError) as e:
        return False


In [19]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            if is_image_file(file_path):
                try:
                    img = Image.open(file_path).resize((64, 64))
                    img_array = np.array(img)
                    label = os.path.basename(subdir)
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading image {file_path}: {e}")
    return images, labels

 
dataset_path = "C:/Users/ninja/Desktop/land cover image classification"
 
images, labels = load_images_from_folder(dataset_path)

 
data = pd.DataFrame({
    'image': images,
    'label': labels
})
 
data = data.sample(frac=1).reset_index(drop=True)

print(data.head())


                                               image        label
0  [[[166, 129, 121], [151, 133, 131], [109, 110,...   Industrial
1  [[[68, 80, 92], [68, 80, 94], [63, 77, 90], [5...  Residential
2  [[[146, 119, 108], [148, 118, 107], [156, 122,...   AnnualCrop
3  [[[105, 109, 154], [99, 112, 157], [90, 106, 1...   Industrial
4  [[[161, 130, 128], [161, 126, 132], [140, 109,...  Residential


In [20]:
data;

In [21]:
data

,image,label
0,"[[[166, 129, 121], [151, 133, 131], [109, 110,...",Industrial
1,"[[[68, 80, 92], [68, 80, 94], [63, 77, 90], [5...",Residential
2,"[[[146, 119, 108], [148, 118, 107], [156, 122,...",AnnualCrop
3,"[[[105, 109, 154], [99, 112, 157], [90, 106, 1...",Industrial
4,"[[[161, 130, 128], [161, 126, 132], [140, 109,...",Residential
...,...,...
26995,"[[[84, 94, 106], [84, 94, 104], [86, 96, 105],...",HerbaceousVegetation
26996,"[[[56, 74, 84], [59, 72, 88], [60, 71, 89], [5...",Industrial
26997,"[[[38, 60, 73], [37, 59, 73], [38, 59, 76], [3...",Forest
26998,"[[[43, 75, 86], [45, 75, 86], [47, 77, 88], [4...",SeaLake


In [27]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

data['image'] = data['image'].apply(lambda x: tf.convert_to_tensor(x, dtype=tf.float32))
label_map = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_map)

train_data, val_data = train_test_split(data, test_size=0.2, random_state=12)


def df_to_dataset(df):
    images = list(df['image'])
    labels = list(df['label'])
    ds = tf.data.Dataset.from_tensor_slices((images, labels))
    ds = ds.batch(128).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

train_dataset = df_to_dataset(train_data)
val_dataset = df_to_dataset(val_data)


In [28]:
from tensorflow.keras import layers, models

In [29]:
 model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(64, 64, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(label_map), activation='softmax')
])
    
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_dataset, validation_data=val_dataset, epochs=16)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()
model.save("land_cover_classification_model.h5")


Epoch 1/16
  9/169 [>.............................] - ETA: 2:53 - loss: 2.2420 - accuracy: 0.1302

KeyboardInterrupt: 